In [ ]:
%pip install setfit

  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=c942309a554cd22fce5ac67546d8cdc2066813603899dc9be31ef83d060abae5
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
!apt install git-lfs

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 24 not upgraded.


In [ ]:
!git config --global user.email "ja_mahmahi@esi.dz"
!git config --global user.name "Anis Mahmahi"

In [ ]:
model_id = "sentence-transformers/paraphrase-mpnet-base-v2"

In [ ]:
import pandas as pd

train = pd.read_csv("/content/train-oana-final-line.csv")
test = pd.read_csv("/content/test-oana-final-line.csv")

In [ ]:
train

,text,technique,article_name
0,Pamela Geller and Robert Spencer co-founded an...,Slogans,111111112
1,"He added: ""We condemn all those whose behaviou...",Black-and-White_Fallacy,111111112
2,"Ms Geller, of the Atlas Shrugs blog, and Mr Sp...",Slogans,111111112
3,On both of their blogs the pair called their b...,Loaded_Language,111111112
4,The UK should never become a stage for inflamm...,Flag-Waving,111111112
...,...,...,...
5383,Also the Left killed comedy.\n,"Exaggeration,Minimisation",999001970
5384,Also the Left killed comedy.\n,Slogans,999001970
5385,That's what Columbia snowflakes thought was of...,"Name_Calling,Labeling",999001970
5386,"Comrades, these jokes you have been listening ...","Exaggeration,Minimisation",999001970


In [ ]:
train = train[train['technique'] != 'Obfuscation,Intentional_Vagueness,Confusion']
test = test[test['technique'] != 'Obfuscation,Intentional_Vagueness,Confusion']

train['technique'] = train['technique'].replace({"Name_Calling,Labeling":"Name_Calling_Labeling"})
test['technique'] = test['technique'].replace({"Name_Calling,Labeling":"Name_Calling_Labeling"})

<ipython-input-7-bd564f531b25>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['technique'] = train['technique'].replace({"Name_Calling,Labeling":"Name_Calling_Labeling"})
<ipython-input-7-bd564f531b25>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['technique'] = test['technique'].replace({"Name_Calling,Labeling":"Name_Calling_Labeling"})


In [ ]:
Appeal_to_authority = ['Appeal_to_Authority']

In [ ]:
b = train[train['technique'].isin(Appeal_to_authority)]
b["G4_label"] = 1


a = train[~train['technique'].isin(Appeal_to_authority)]
a["G4_label"] = 0


G4_data = pd.concat([b, a], ignore_index=True)
G4_data

<ipython-input-10-c98e8e69e74c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  b["G4_label"] = 1
<ipython-input-10-c98e8e69e74c>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  a["G4_label"] = 0


,text,technique,article_name,G4_label
0,As the political scientist Richard Neustadt sa...,Appeal_to_Authority,111111122,1
1,“I can tell you 100% this is not that kind of ...,Appeal_to_Authority,694811415,1
2,"Now, new information released by investigative...",Appeal_to_Authority,695833178,1
3,"According to Loomer, she obtained the image fr...",Appeal_to_Authority,695833178,1
4,Pope Francis explains this again in AL 305 whe...,Appeal_to_Authority,697454736,1
...,...,...,...,...
5372,Also the Left killed comedy.\n,"Exaggeration,Minimisation",999001970,0
5373,Also the Left killed comedy.\n,Slogans,999001970,0
5374,That's what Columbia snowflakes thought was of...,Name_Calling_Labeling,999001970,0
5375,"Comrades, these jokes you have been listening ...","Exaggeration,Minimisation",999001970,0


In [ ]:
from sklearn.utils import resample

# Separate majority and minority classes
df_majority = G4_data[G4_data['G4_label'] == 0]
df_minority = G4_data[G4_data['G4_label'] == 1]

# Downsample the majority class
df_majority_downsampled = resample(df_majority,
                                   replace=False,  # Sample without replacement
                                   n_samples=len(df_minority)*4,  # Match minority class size
                                   )  # For reproducibility

G4_data = pd.concat([df_majority_downsampled, df_minority])
df4 = G4_data[['text', 'G4_label' ]]
df4= df4.rename(columns={'G4_label': 'label'})
df4

,text,label
3917,“They know this is one of the great scandals i...,0
4562,The Guardian did not make a mistake in vilifyi...,0
2740,He himself said: “No one defends Islam like Ar...,0
2303,"For example, he quotes 5:51 -- “You who believ...",0
5232,"She is a hijab-wearing, devout Muslim, and Isl...",0
...,...,...
108,"The government needs, he said, to “implement a...",1
109,And he said it needs to affirm “Shariah’s assa...,1
110,"When asked for closing remarks, Patel responde...",1
111,As The American Mirror’s Kyle Olsen concludes ...,1


In [ ]:
b_2 = test[test['technique'].isin(Appeal_to_authority)]
b_2["G4_label"] = 1


a_2 = test[~test['technique'].isin(Appeal_to_authority)]
a_2["G4_label"] = 0


test = pd.concat([b_2, a_2], ignore_index=True)
test

<ipython-input-13-782f922f147d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  b_2["G4_label"] = 1
<ipython-input-13-782f922f147d>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  a_2["G4_label"] = 0


,text,technique,article_name,G4_label
0,"""The next transmission could be more pronounce...",Appeal_to_Authority,111111111,1
1,This along with poor sanitation and vector con...,Appeal_to_Authority,111111111,1
2,"In a statement following that accusation, Fran...",Appeal_to_Authority,111111115,1
3,"Edward E. Curtis IV, the author of “Black Musl...",Appeal_to_Authority,111111135,1
4,Madagascar Hospitals On High Alert: ‘No One Is...,Appeal_to_Authority,700387229,1
...,...,...,...,...
1298,Acosta ignorantly blathered on to reporters wh...,"Exaggeration,Minimisation",999001419,0
1299,The Acosta-CNN lawsuit was filed by Ted Boutro...,Name_Calling_Labeling,999001419,0
1300,"The yanking of the physical credential, by the...",Loaded_Language,999001419,0
1301,"The president described Acosta as ""unbelievabl...",Name_Calling_Labeling,999001419,0


In [ ]:
test.rename(columns = {'G4_label':'label'}, inplace=True)
test[['text', 'label']].to_csv("final_test.csv", index=False)
df4[['text', 'label']].to_csv("final_train.csv", index=False)

In [ ]:
test

,text,technique,article_name,label
0,"""The next transmission could be more pronounce...",Appeal_to_Authority,111111111,1
1,This along with poor sanitation and vector con...,Appeal_to_Authority,111111111,1
2,"In a statement following that accusation, Fran...",Appeal_to_Authority,111111115,1
3,"Edward E. Curtis IV, the author of “Black Musl...",Appeal_to_Authority,111111135,1
4,Madagascar Hospitals On High Alert: ‘No One Is...,Appeal_to_Authority,700387229,1
...,...,...,...,...
1298,Acosta ignorantly blathered on to reporters wh...,"Exaggeration,Minimisation",999001419,0
1299,The Acosta-CNN lawsuit was filed by Ted Boutro...,Name_Calling_Labeling,999001419,0
1300,"The yanking of the physical credential, by the...",Loaded_Language,999001419,0
1301,"The president described Acosta as ""unbelievabl...",Name_Calling_Labeling,999001419,0


In [ ]:
from datasets import load_dataset


dataset = load_dataset('csv', data_files={'train': "/content/final_train.csv",'test': "/content/final_test.csv"}, )
dataset

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 565
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 1303
    })
})

In [ ]:
from setfit import SetFitModel
from sentence_transformers import SentenceTransformer
from sklearn.linear_model import LogisticRegression

model_body=SentenceTransformer("sentence-transformers/paraphrase-mpnet-base-v2")
model_head=LogisticRegression(class_weight="balanced")
model = SetFitModel(model_body=model_body, model_head=model_head,labels=[0,1],)


# # Load a SetFit model from Hub
# model = SetFitModel.from_pretrained(
#     "sentence-transformers/paraphrase-mpnet-base-v2",
#     labels=[0,1],
# )


In [ ]:
model.to("cuda")

SetFitModel(model_body=SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: MPNetModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
), model_head=LogisticRegression(class_weight='balanced'), multi_target_strategy=None, normalize_embeddings=False, labels=[0, 1], model_card_data=SetFitModelCardData(language=None, license=None, tags=['setfit', 'sentence-transformers', 'text-classification', 'generated_from_setfit_trainer'], model_name='SetFit', model_id=None, dataset_name=None, dataset_id=None, dataset_revision=None, task_name=None, st_id=None, hyperparameters={}, eval_results_dict={}, eval_lines_list=[], metric_lines=[], widget=[], predict_example=None, label_example_list=[], tokenizer_warning=False, train_set_metrics_list=[], train_set_sentences_per_label_list=[], code_carbon_cal

In [ ]:
from setfit import SetFitModel, Trainer, TrainingArguments, sample_dataset

args = TrainingArguments(
    batch_size=16,
    num_epochs=2,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    num_iterations=20,
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=dataset['train'],
    eval_dataset=dataset['test'],
    metric="f1",
    column_mapping={"text": "text", "label": "label"}  # Map dataset columns to text/label expected by trainer
)


Applying column mapping to the training dataset
Applying column mapping to the evaluation dataset


In [ ]:
import torch
training_device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
training_device

device(type='cuda')

In [ ]:
# Train and evaluate
trainer.train()


***** Running training *****
  Num examples = 1413
  Num epochs = 2
  Total optimization steps = 2826
  Total train batch size = 16


Epoch,Training Loss,Validation Loss,Embedding Loss,Rate
1,No log,No log,0.286800,0.000011
2,No log,No log,0.292600,0.000000


  0%|          | 0/3258 [00:00<?, ?it/s]

  0%|          | 0/3258 [00:00<?, ?it/s]

Loading best SentenceTransformer model from step 1413.


In [ ]:
metrics = trainer.evaluate(dataset['test'])
print(metrics)


Applying column mapping to the evaluation dataset
***** Running evaluation *****


{'f1': 0.20795107033639143}


In [ ]:
trainer.push_to_hub("anismahmahi/appeal-to-authority-setfit-model")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

model_head.pkl:   0%|          | 0.00/6.99k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/anismahmahi/appeal-to-authority-setfit-model/commit/90fbb1e40ec5976d079293809f90d2fdb96e890a', commit_message='Add SetFit model', commit_description='', oid='90fbb1e40ec5976d079293809f90d2fdb96e890a', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
from sklearn.metrics import f1_score,classification_report
preds = model.predict(list(test['text']))



In [ ]:
print(classification_report(test['label'],preds))

              precision    recall  f1-score   support

           0       0.99      0.80      0.89      1256
           1       0.12      0.72      0.21        47

    accuracy                           0.80      1303
   macro avg       0.55      0.76      0.55      1303
weighted avg       0.96      0.80      0.86      1303

